In [1]:
%pip install pandas requests

Note: you may need to restart the kernel to use updated packages.


10 rows trial

In [ ]:
import pandas as pd
import requests
import time

df_full = pd.read_csv('output_cleaned.csv')  # original full dataset
df = df_full.head(10)  # only take first 10 rows

API_KEY = "API_KEY"  # replace this  

def reverse_geocode(lat, lng):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={API_KEY}'
    response = requests.get(url).json()
    if response['status'] == 'OK' and response['results']:
        return response['results'][0]['formatted_address']
    return None

def geocode(address):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}'
    response = requests.get(url).json()
    if response['status'] == 'OK' and response['results']:
        location = response['results'][0]['geometry']['location']
        return location['lat'], location['lng']
    return None, None

df['reverse_geocoded_address'] = df.apply(
    lambda row: reverse_geocode(row['latitude'], row['longitude']), axis=1
)
time.sleep(1)

df[['new_latitude', 'new_longitude']] = df['reverse_geocoded_address'].apply(
    lambda addr: pd.Series(geocode(addr)) if pd.notnull(addr) else pd.Series([None, None])
)
df.to_csv('output_test_10rows.csv', index=False)
print("✅ File saved: output_test_10rows.csv")


/var/folders/y5/p42j4t91737_2v19wfkt3sv80000gn/T/ipykernel_8700/317866638.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reverse_geocoded_address'] = df.apply(


✅ File saved: output_test_10rows.csv


/var/folders/y5/p42j4t91737_2v19wfkt3sv80000gn/T/ipykernel_8700/317866638.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['new_latitude', 'new_longitude']] = df['reverse_geocoded_address'].apply(
/var/folders/y5/p42j4t91737_2v19wfkt3sv80000gn/T/ipykernel_8700/317866638.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['new_latitude', 'new_longitude']] = df['reverse_geocoded_address'].apply(


reverse geocoding entire cleaned document

In [ ]:
import pandas as pd
import requests
import time

df = pd.read_csv('output_cleaned.csv')  # Replace with your full dataset

API_KEY = "API_KEY"  # replace this  

def reverse_geocode(lat, lng):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={API_KEY}'
    response = requests.get(url).json()
    if response['status'] == 'OK' and response['results']:
        return response['results'][0]['formatted_address']
    return None

def geocode(address):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}'
    response = requests.get(url).json()
    if response['status'] == 'OK' and response['results']:
        location = response['results'][0]['geometry']['location']
        return location['lat'], location['lng']
    return None, None


df.loc[:, 'reverse_geocoded_address'] = df.apply(
    lambda row: reverse_geocode(row['latitude'], row['longitude']), axis=1
)
time.sleep(1)  # Optional rate limit pause

df[['new_latitude', 'new_longitude']] = df['reverse_geocoded_address'].apply(
    lambda addr: pd.Series(geocode(addr)) if pd.notnull(addr) else pd.Series([None, None])
)

df.to_csv('output_with_geocoding.csv', index=False)
print("✅ Full file saved: output_with_geocoding.csv")


✅ Full file saved: output_with_geocoding.csv


In [5]:
%pip install sentence-transformers hdbscan umap-learn scikit-learn

Note: you may need to restart the kernel to use updated packages.


HDBSCAN clustering match

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import hdbscan
import numpy as np

df = pd.read_csv("output_with_geocoding.csv")
df['address'] = df['address'].fillna('').astype(str)
df['reverse_geocoded_address'] = df['reverse_geocoded_address'].fillna('').astype(str)

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

print("Generating embeddings...")
embed_orig = model.encode(df['address'].tolist(), show_progress_bar=True)
embed_rev = model.encode(df['reverse_geocoded_address'].tolist(), show_progress_bar=True)

print("Reducing with UMAP...")
umap_model = UMAP(n_neighbors=15, n_components=15, metric='cosine')
embed_orig_reduced = umap_model.fit_transform(embed_orig)
embed_rev_reduced = umap_model.fit_transform(embed_rev)

print("Clustering original addresses...")
cluster_orig = hdbscan.HDBSCAN(min_cluster_size=3, min_samples=1, metric='euclidean').fit_predict(embed_orig_reduced)

print("Clustering reverse-geocoded addresses...")
cluster_rev = hdbscan.HDBSCAN(min_cluster_size=3, min_samples=1, metric='euclidean').fit_predict(embed_rev_reduced)

df['HDBSCAN_cluster_address'] = cluster_orig
df['HDBSCAN_cluster_reverse'] = cluster_rev
df['cluster_match'] = df['HDBSCAN_cluster_address'] == df['HDBSCAN_cluster_reverse']

print("Calculating cosine similarity between address pairs...")
cos_sim = cosine_similarity(embed_orig, embed_rev)
df['cosine_similarity'] = np.diag(cos_sim)

df.to_csv("address_cluster_comparison.csv", index=False)
print("✅ Output saved to address_cluster_comparison.csv")
print(f"✔️ Matching clusters: {df['cluster_match'].sum()} out of {len(df)}")


Generating embeddings...


Batches:   0%|          | 0/115 [00:00<?, ?it/s]

Batches:   0%|          | 0/115 [00:00<?, ?it/s]

Reducing with UMAP...
Clustering original addresses...
Clustering reverse-geocoded addresses...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Calculating cosine similarity between address pairs...
✅ Output saved to address_cluster_comparison.csv
✔️ Matching clusters: 163 out of 3658


TF-IDF cosine similairty match

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("output_with_geocoding.csv")

df['address'] = df['address'].fillna('').astype(str)
df['reverse_geocoded_address'] = df['reverse_geocoded_address'].fillna('').astype(str)

all_addresses = df['address'].tolist() + df['reverse_geocoded_address'].tolist()

# Fit TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Use unigrams and bigrams
tfidf_matrix = vectorizer.fit_transform(all_addresses)

# Split the TF-IDF back
original_tfidf = tfidf_matrix[:len(df)]
reverse_tfidf = tfidf_matrix[len(df):]

# Compute pairwise cosine similarity
similarities = cosine_similarity(original_tfidf, reverse_tfidf)

# Extract diagonal elements (same-row comparisons)
df['tfidf_cosine_sim'] = [similarities[i, i] for i in range(len(df))]

# Optional: Flag matches above a threshold
df['likely_match'] = df['tfidf_cosine_sim'] >= 0.75

print(df[['address', 'reverse_geocoded_address', 'tfidf_cosine_sim', 'likely_match']].head())
print(f"\n✔️ Matches above 0.75 similarity: {df['likely_match'].sum()} out of {len(df)}")

df.to_csv("address_TF-IDF_comparison.csv", index=False)
print("✅ Output saved to address_TF-IDF_comparison.csv")


                                           address  \
0   atur park basant garden chembur mumbai  400071   
1          tilak road opp bikaji ghatkopare 400077   
2    station road vikhroliwest station road 400083   
3      oppshreyas cinema lbsmarg ghatkoparw 400082   
4                    nr bhaji mandi kurlaw  400070   

                            reverse_geocoded_address  tfidf_cosine_sim  \
0  Shop No 11, 12, Sion - Trombay Rd, opp. Maitri...          0.507229   
1  Shop No.4, Neelkanth Sadan, Hingwala Ln, opp. ...          0.049153   
2  5, Station Rd, near Laxmi dairy, Lokamanya Nag...          0.086518   
3  1, Lal Bahadur Shastri Marg, Sainath Nagar, Ni...          0.000000   
4  Shop No 43 laxman road yadav market kurla bhaj...          0.066684   

   likely_match  
0         False  
1         False  
2         False  
3         False  
4         False  

✔️ Matches above 0.75 similarity: 46 out of 3658
✅ Output saved to address_TF-IDF_comparison.csv
